# description
YYYYMMDD = Datum (YYYY=jaar MM=maand DD=dag);<br>
DDVEC = Vectorgemiddelde windrichting in graden (360=noord, 90=oost, 180=zuid, 270=west, 0=windstil/variabel). Zie http://www.knmi.nl/kennis-en-datacentrum/achtergrond/klimatologische-brochures-en-boeken; <br>
FHVEC = Vectorgemiddelde windsnelheid (in 0.1 m/s). Zie http://www.knmi.nl/kennis-en-datacentrum/achtergrond/klimatologische-brochures-en-boeken; <br>
FG = Etmaalgemiddelde windsnelheid (in 0.1 m/s);<br>
FHX = Hoogste uurgemiddelde windsnelheid (in 0.1 m/s);<br> 
FHXH = Uurvak waarin FHX is gemeten; <br>
FHN = Laagste uurgemiddelde windsnelheid (in 0.1 m/s);<br>
FHNH = Uurvak waarin FHN is gemeten; <br>
FXX = Hoogste windstoot (in 0.1 m/s); <br>
FXXH = Uurvak waarin FXX is gemeten; <br>
TG = Etmaalgemiddelde temperatuur (in 0.1 graden Celsius);<br> 
TN = Minimum temperatuur (in 0.1 graden Celsius);<br>
TNH = Uurvak waarin TN is gemeten; <br>
TX = Maximum temperatuur (in 0.1 graden Celsius);<br>
TXH = Uurvak waarin TX is gemeten; <br>
T10N = Minimum temperatuur op 10 cm hoogte (in 0.1 graden Celsius);<br> 
T10NH = 6-uurs tijdvak waarin T10N is gemeten; 6=0-6 UT, 12=6-12 UT, 18=12-18 UT, 24=18-24 UT;<br>
SQ = Zonneschijnduur (in 0.1 uur) berekend uit de globale straling (-1 voor <0.05 uur);<br>
SP = Percentage van de langst mogelijke zonneschijnduur;<br>
Q = Globale straling (in J/cm2);<br>
DR = Duur van de neerslag (in 0.1 uur);<br>
RH = Etmaalsom van de neerslag (in 0.1 mm) (-1 voor <0.05 mm);<br>
RHX = Hoogste uursom van de neerslag (in 0.1 mm) (-1 voor <0.05 mm);<br> 
RHXH = Uurvak waarin RHX is gemeten;<br>
PG = Etmaalgemiddelde luchtdruk herleid tot zeeniveau (in 0.1 hPa) berekend uit 24 uurwaarden;<br>
PX = Hoogste uurwaarde van de luchtdruk herleid tot zeeniveau (in 0.1 hPa);<br>
PXH = Uurvak waarin PX is gemeten;<br>
PN = Laagste uurwaarde van de luchtdruk herleid tot zeeniveau (in 0.1 hPa);<br> 
PNH = Uurvak waarin PN is gemeten; <br>
VVN = Minimum opgetreden zicht; 0: <100 m, 1:100-200 m, 2:200-300 m,..., 49:4900-5000 m, 50:5-6 km, 56:6-7 km, 57:7-8 km,..., 79:29-30 km, 80:30-35 km, 81:35-40 km,..., 89: >70 km);<br>
VVNH = Uurvak waarin VVN is gemeten; <br>
VVX = Maximum opgetreden zicht; 0: <100 m, 1:100-200 m, 2:200-300 m,..., 49:4900-5000 m, 50:5-6 km, 56:6-7 km, 57:7-8 km,..., 79:29-30 km, 80:30-35 km, 81:35-40 km,..., 89: >70 km);<br>
VVXH = Uurvak waarin VVX is gemeten; <br>
NG = Etmaalgemiddelde bewolking (bedekkingsgraad van de bovenlucht in achtsten, 9=bovenlucht onzichtbaar); <br>
UG = Etmaalgemiddelde relatieve vochtigheid (in procenten); <br>
UX = Maximale relatieve vochtigheid (in procenten); <br>
UXH = Uurvak waarin UX is gemeten; <br>
UN = Minimale relatieve vochtigheid (in procenten); <br>
UNH = Uurvak waarin UN is gemeten; <br>
EV24 = Referentiegewasverdamping (Makkink) (in 0.1 mm);<br>

In [17]:
# http://projects.knmi.nl/klimatologie/daggegevens/selectie.cgi

import pandas as pd
import numpy as np

df = pd.read_csv('data/weather.csv', low_memory=False)
df1 = pd.read_csv('data/location.csv')
df['STN'] = df['STN'].map(df1.set_index('STN')['NAME'])

df

,STN,YYYYMMDD,DDVEC,FHVEC,FG,FHX,FHXH,FHN,FHNH,FXX,...,VVNH,VVX,VVXH,NG,UG,UX,UXH,UN,UNH,EV24
0,IJMOND,20010130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,IJMOND,20010131,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,IJMOND,20010201,236.0,13.0,24.0,50.0,8.0,10.0,1.0,90.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,IJMOND,20010202,150.0,46.0,58.0,80.0,9.0,10.0,3.0,100.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,IJMOND,20010203,106.0,65.0,66.0,90.0,8.0,50.0,1.0,120.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347476,ARCEN,20191206,215.0,56.0,57.0,70.0,20.0,40.0,4.0,140.0,...,NaN,NaN,NaN,NaN,94.0,97.0,10.0,87.0,4.0,1.0
347477,ARCEN,20191207,236.0,40.0,42.0,50.0,1.0,30.0,18.0,100.0,...,NaN,NaN,NaN,NaN,86.0,96.0,2.0,79.0,19.0,2.0
347478,ARCEN,20191208,226.0,58.0,59.0,80.0,7.0,40.0,1.0,170.0,...,NaN,NaN,NaN,NaN,81.0,96.0,10.0,73.0,15.0,2.0
347479,ARCEN,20191209,245.0,34.0,40.0,60.0,1.0,20.0,15.0,130.0,...,NaN,NaN,NaN,NaN,87.0,97.0,9.0,77.0,3.0,2.0
